In [32]:
s1 = '说起鸿蒙系统，相信大家都非常熟悉了，这个系统自2019年推出，是华为被打压之后进行的反击。'
s2 = '而预约23999元的MateBook Fold 非凡大师的人数，超过了13万，就按23999元来算，这里就是31亿多，合计就是18万人预约，如果最终这些电脑都成交了，相当于销售金额就超过了35亿元。'
s3 = '为何这以多人预约鸿蒙电脑？一方面当然是因为华为的号召力，不管是MateBook Pro，还是MateBook Fold 非凡大师都是产品力非常棒的产品，自然有人买，特别是折叠屏电脑，更是创新十足，噱头十足。'
len(s1),len(s2),len(s3)

(45, 99, 103)

In [33]:
# tokenizer
import jieba

# <UNK> 默认填充符号，当遇到不认识的词时，使用<UNK>填充
# <PAD> 填充符号，当句子长度不够时，使用<PAD>填充
words = {"<UNK>", "<PAD>"}
for sentence in [s1, s2, s3]:
    words = words.union(set(jieba.lcut(sentence)))

# 字典
word2idx = {word: idx for idx, word in enumerate(words)}
idx2word = {idx: word for word, idx in word2idx.items()}

# 把每句话进行token化
s1_idx = [word2idx.get(word, word2idx.get("<UNK>")) for word in jieba.lcut(s1)]
s2_idx = [word2idx.get(word, word2idx.get("<UNK>")) for word in jieba.lcut(s2)]
s3_idx = [word2idx.get(word, word2idx.get("<UNK>")) for word in jieba.lcut(s3)]

len(s1_idx),len(s2_idx),len(s3_idx)


(27, 53, 54)

In [34]:
# 在早期，文本处理可以使用CNN来进行，后来，又有人用RNN来处理，当然，现在这些都不怎么用了，已经被先进的Transformer架构所取代了。
# CNN处理NLP任务
"""
CNN卷积网络
是用来处理图像的
现在用来处理文本有一些前提条件
训练的时候需要把训练集文本处理成长度大小一致的文本，通过填充或者截断
缺点就是会引入噪声和丢失信息
"""
# 长度处理成一致的：全部处理成长度=53
s1_idx2 = s1_idx + (len(s2_idx) - len(s1_idx)) * [word2idx.get("<PAD>")]
s2_idx2 = s2_idx.copy()
s3_idx2 = s3_idx[:len(s2_idx)]

len(s1_idx2),len(s2_idx2),len(s3_idx2)

(53, 53, 53)

In [39]:
# 转张量
import torch
from torch import nn

s_vec = torch.tensor(data = [s1_idx2, s2_idx2, s3_idx2], dtype=torch.long)

torch.Size([3, 53])

In [40]:
s_vec.shape

torch.Size([3, 53])

In [41]:
s_vec

tensor([[59, 22, 37,  9, 40, 66, 74, 18,  4, 16,  9, 47, 37, 48, 26, 56, 54,  9,
         58,  3, 80, 72, 79, 31, 42, 14, 39,  7,  7,  7,  7,  7,  7,  7,  7,  7,
          7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7],
        [ 1, 55, 62, 68, 42, 50, 81, 41, 81, 67, 75, 42, 70,  9, 15, 16, 23, 44,
          9, 57, 60, 62, 77,  2,  9, 28, 63, 27, 46,  9, 13, 63, 61,  0, 55,  9,
         76, 78, 51, 30, 74, 29, 16,  9, 52, 43, 12, 57, 15, 16, 73, 49, 39],
        [69, 10, 65, 20, 55, 22, 30, 25, 83, 82, 64,  3, 42, 38,  9, 21, 58, 50,
         81, 17,  9, 53, 50, 81, 41, 81, 67, 75, 74, 58, 84, 32, 18, 19, 42, 84,
          9, 71, 24,  6,  9,  5, 58, 33, 35, 30,  9, 11, 36, 45,  9, 34, 45]])

In [52]:
# Embedding
seq_len = len(s1_idx2)
dict_len = len(word2idx)
embedding_dim = 32
print(f"seq_len: {seq_len}, dict_len: {dict_len}, embedding_dim: {embedding_dim}")
# ont-hot编码
# [1, 0, 0, ...]
# [0, 1, 0, ...]
# [0, 0, 1, ...]
# 因为one-hot编码，占空间，稀疏，不包含语义信息等等，基本都不用了。所以都用Embedding

# 看我们的3个句子，每个句子长度是53，字典的长度是85，shape就是 [3, 53, 85]
# Embedding可以降维，比如降维到 32； 就是把上面的矩阵[3, 53, 85]乘以一个降维矩阵[85,32]，结果就是[3, 53, 32]
# 这是从数学角度来看的，实际上Embedding都给我们处理好了，直接调用API就行
embed = nn.Embedding(num_embeddings=dict_len, 
                     embedding_dim=embedding_dim,
                     padding_idx=word2idx.get("<PAD>"))

# Embedding
vec = embed(s_vec)

print('vec.shape:', vec.shape)

seq_len: 53, dict_len: 85, embedding_dim: 32
vec.shape: torch.Size([3, 53, 32])


In [54]:
# 在早期，文本处理可以使用CNN来进行，后来，又有人用RNN来处理，当然，现在这些都不怎么用了，已经被先进的Transformer架构所取代了。
# CNN处理NLP任务
# 构建CNN卷积网络
class Conv1Block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # 这里用的是 1d 1维卷积
        self.conv1d = nn.Conv1d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=3,
                               padding=1)
        self.bn = nn.BatchNorm1d(num_features=out_channels)
        
    
    def forward(self, x):
        x = self.conv1d(x)
        x = self.bn(x)
        x = torch.relu(x)
        return x

In [58]:
# 构建模型
class Model(nn.Module):
    def __init__(self, dict_len, embedding_dim=256, n_classes=2):
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=dict_len, 
                                 embedding_dim=embedding_dim,
                                 padding_idx=word2idx.get("<PAD>"))
        self.conv1 = Conv1Block(in_channels=embedding_dim,
                               out_channels=2 * embedding_dim)
        self.conv2 = Conv1Block(in_channels=2 * embedding_dim,
                               out_channels=4 * embedding_dim)
        self.conv3 = Conv1Block(in_channels=4 * embedding_dim,
                               out_channels=8 * embedding_dim)
        self.conv4 = Conv1Block(in_channels=8 * embedding_dim,
                               out_channels=16 * embedding_dim)
        self.mp = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        
        # 这里的要计算一下
        self.fc1 = nn.Linear(in_features=45056,
                           out_features=512)
        self.fc2 = nn.Linear(in_features=512,
                            out_features=n_classes)
    
    def forward(self, x):
        # 先进行Embedding
        x = self.embed(x)
        # 维度转换：因为卷积要求的输入是[N,C,L]，我们现在是[N,L,C]所以要经过permute转换
        x = torch.permute(input=x, dims=(0, 2, 1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.mp(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.mp(x)
        # 拉平，也可以使用flatten
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x        

In [61]:
# 测试模型
# 创建模型:这里是mock的数据，并不是用我们之前处理的s1,s2,s3那三个句子
model = Model(dict_len=5000, embedding_dim=256, n_classes=2)
# 构建测试数据
X = torch.randint(low=0, high=5001, size=(3, 46))
# 训练
y_pred = model(X)
# 查看模型返回结果
y_pred.shape

torch.Size([3, 2])